In [ ]:
#!pip install transformers==4.8.0

In [ ]:
#!pip install torch==1.9.0
#!conda install -c conda-forge torch==1.9.0

In [28]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import transformers

In [29]:
torch.__version__, transformers.__version__

('1.10.2', '4.8.0')

In [30]:
DATA_FOLDER = 'data/rel_ext_data/'

In [4]:
df_train = pd.read_csv(DATA_FOLDER +'train.csv', encoding='utf-8')
df_train.head()

,text,intent
0,tell me the expiration date for my current cre...,expiration_date
1,would you disconnect from my phone,sync_device
2,could you please track my package,order_status
3,any travel alerts for canada,travel_alert
4,i want to report fraudulent activity on my ame...,report_fraud


In [5]:
df_val = pd.read_csv(DATA_FOLDER +'val.csv', encoding='utf-8')
df_val.head()

,text,intent
0,hey,greeting
1,put laundry on my chore list,todo_list_update
2,go into whisper mode,whisper_mode
3,when do i need to change my motor oil again,oil_change_when
4,what is the insurance plan i am enrolled in,insurance


In [6]:
df_test = pd.read_csv(DATA_FOLDER +'test.csv', encoding='utf-8')
df_test.head()

,text,intent
0,thanks so much ai,thank_you
1,i will be traveling to lima alert my bank,travel_notification
2,say again please,repeat
3,what is needed to cook lasagna,ingredients_list
4,give me the pressure for the tires on my car,tire_pressure


In [31]:
labels = {k:i for i,k in enumerate(set(df_train['intent'].tolist()))}
labels

{'change_speed': 0,
 'share_location': 1,
 'change_user_name': 2,
 'shopping_list_update': 3,
 'w2': 4,
 'how_busy': 5,
 'card_declined': 6,
 'next_holiday': 7,
 'rewards_balance': 8,
 'pin_change': 9,
 'pto_used': 10,
 'new_card': 11,
 'meeting_schedule': 12,
 'roll_dice': 13,
 'flight_status': 14,
 'goodbye': 15,
 'credit_limit_change': 16,
 'damaged_card': 17,
 'ingredients_list': 18,
 'insurance_change': 19,
 'pto_request': 20,
 'cancel_reservation': 21,
 'spelling': 22,
 'timezone': 23,
 'greeting': 24,
 'who_do_you_work_for': 25,
 'tell_joke': 26,
 'transactions': 27,
 'apr': 28,
 'car_rental': 29,
 'oil_change_how': 30,
 'international_visa': 31,
 'yes': 32,
 'uber': 33,
 'min_payment': 34,
 'play_music': 35,
 'user_name': 36,
 'nutrition_info': 37,
 'expiration_date': 38,
 'time': 39,
 'vaccines': 40,
 'meal_suggestion': 41,
 'redeem_rewards': 42,
 'last_maintenance': 43,
 'restaurant_suggestion': 44,
 'application_status': 45,
 'what_song': 46,
 'current_location': 47,
 'gas_t

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['intent']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [33]:
torch.cuda.is_available()

True

In [34]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 150)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [44]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.type(torch.LongTensor) # <---- Here (casting)

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:
                    val_label = val_label.type(torch.LongTensor) # <---- Here (casting)

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [45]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

In [ ]:
BATCH_SIZE = 1
EPOCHS = 10
model = BertClassifier()
LR = 1e-3
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 7500/7500 [28:19<00:00,  4.41it/s]


Epochs: 1 | Train Loss:  5.011 | Train Accuracy:  0.006 | Val Loss:  5.011 | Val Accuracy:  0.007


100%|██████████████████████████████████████████████████████████████████████████████| 7500/7500 [28:18<00:00,  4.42it/s]


Epochs: 2 | Train Loss:  5.011 | Train Accuracy:  0.007 | Val Loss:  5.011 | Val Accuracy:  0.007


  6%|█████                                                                          | 479/7500 [01:48<26:28,  4.42it/s]

In [64]:
1e-3

0.001

In [ ]:
evaluate(model, df_test)